# Create object LBP signatures - after frequency counted has been made into numpy
This code attempts to combine the LBP numbers for (by doing a frequency count across them) individual object (simulated nuclei). This one is doing this with one property difference

In [53]:
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
#import cv2
import skimage
import tifffile
import os
import pandas as pd
#import napari
#print(napari.__version__)
#from dask_image.imread import imread
#import dask.array as da
#import io

import platform
if platform.system() == 'Windows':
    fileroot = 'X:/'
    print('Loading Windows')
if platform.system() == 'Linux':
    fileroot = '/home/jovyan/'
    print('Loading Linux')
if os.path.exists('/home/ubuntu/Documents/'):
    fileroot = '/home/ubuntu/Documents/'
if os.path.exists('/nfs/team283_imaging/') and not os.path.exists('/home/jovyan/'):
    fileroot = '/nfs/team283_imaging/'

colors_new=['#0072b2','#d55e00','#009e73', '#cc79a7','#f0e442','#56b4e9']

import sys
sys.path.append(fileroot+'VU_TEX/Ben_utilities/')
import useful_functions as uf
import numba_funcs as nf

Loading Linux


In [54]:
fileroot

'/home/jovyan/'

In [55]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [56]:
maxi = 3328 #this is the maximum value that the LBP could get to

In [57]:
from tqdm import tqdm
from scipy import ndimage as ndi
from functools import partial
from copy import deepcopy
from itertools import product
from datetime import datetime
from tqdm.notebook import trange, tqdm
import anndata as ad

### Loading the data

In [58]:
patchsize = 200

In [59]:
directory_originals = [fileroot + '0ExternalData/2022-09-01_IVY_GAP/2022-08-31_originals/']
#directory_originals = [fileroot + 'team283_imaging/0ExternalData/2022-09-01_IVY_GAP/2022-08-31_originals/']
filenames_originals = ['donor_id_12106_specimen_id_268416305_subimage_id_273294441.npy',
 'donor_id_292023109_specimen_id_293252159_subimage_id_294219224.npy',
 'donor_id_12165_specimen_id_706710_subimage_id_101713024.npy',
 'donor_id_15343_specimen_id_281857486_subimage_id_286688001.npy',
 'donor_id_12112_specimen_id_299863542_subimage_id_300630264.npy',
 'donor_id_12111_specimen_id_301589924_subimage_id_302049407.npy',
 'donor_id_12111_specimen_id_287469118_subimage_id_292327668.npy',
 'donor_id_12108_specimen_id_298804531_subimage_id_309715132.npy',
 'donor_id_12165_specimen_id_706867_subimage_id_300631309.npy',
 'donor_id_13988_specimen_id_278113164_subimage_id_286689303.npy',
 'donor_id_12111_specimen_id_310607296_subimage_id_311174531.npy',
 'donor_id_12165_specimen_id_287815403_subimage_id_292344018.npy']
#filenames_originals = os.listdir(directory_originals[0])
filenames_originals = [file.replace('.npy', '.jpg') for file in filenames_originals]
#fullpaths_originals = [x + y for x,y in zip(directory_originals, filenames_originals)]
fullpaths_originals = [directory_originals[0] + y for y in filenames_originals if os.path.exists(directory_originals[0] + y)]
fullpaths_originals

['/home/jovyan/0ExternalData/2022-09-01_IVY_GAP/2022-08-31_originals/donor_id_12106_specimen_id_268416305_subimage_id_273294441.jpg',
 '/home/jovyan/0ExternalData/2022-09-01_IVY_GAP/2022-08-31_originals/donor_id_292023109_specimen_id_293252159_subimage_id_294219224.jpg',
 '/home/jovyan/0ExternalData/2022-09-01_IVY_GAP/2022-08-31_originals/donor_id_12165_specimen_id_706710_subimage_id_101713024.jpg',
 '/home/jovyan/0ExternalData/2022-09-01_IVY_GAP/2022-08-31_originals/donor_id_15343_specimen_id_281857486_subimage_id_286688001.jpg',
 '/home/jovyan/0ExternalData/2022-09-01_IVY_GAP/2022-08-31_originals/donor_id_12112_specimen_id_299863542_subimage_id_300630264.jpg',
 '/home/jovyan/0ExternalData/2022-09-01_IVY_GAP/2022-08-31_originals/donor_id_12111_specimen_id_301589924_subimage_id_302049407.jpg',
 '/home/jovyan/0ExternalData/2022-09-01_IVY_GAP/2022-08-31_originals/donor_id_12111_specimen_id_287469118_subimage_id_292327668.jpg',
 '/home/jovyan/0ExternalData/2022-09-01_IVY_GAP/2022-08-31_or

In [60]:
directory_lbp_top = fileroot + 'VU_TEX/playground_Ben/2022-08-26_IVY_GAP/2022-11-09_Subset_optimisation_LBP_v18_patchsize_200/B_LBP_full_images/'
print(directory_lbp_top)
#should be aligned with fullpaths_originals

/home/jovyan/VU_TEX/playground_Ben/2022-08-26_IVY_GAP/2022-11-09_Subset_optimisation_LBP_v18_patchsize_200/B_LBP_full_images/


In [61]:
#directory_gt = fileroot + 'team283_imaging/0ExternalData/2022-09-01_IVY_GAP/2022-09-02_integer_annotations/'
directory_gt = fileroot + 'VU_TEX/playground_Ben/2022-08-26_IVY_GAP/2022-11-09_Subset_optimisation_LBP_v18_patchsize_200/2022-11-09_integer_annotations_binned_patchsize_200/'
#filename_gts = os.listdir(directory_gt)
filename_gts = [each.replace('.jpg', '_annotation.npy') for each in filenames_originals]
fullpaths_gts = [directory_gt + filename_gt for filename_gt in filename_gts if os.path.exists(directory_gt + filename_gt)]
fullpaths_gts

['/home/jovyan/VU_TEX/playground_Ben/2022-08-26_IVY_GAP/2022-11-09_Subset_optimisation_LBP_v18_patchsize_200/2022-11-09_integer_annotations_binned_patchsize_200/donor_id_12106_specimen_id_268416305_subimage_id_273294441_annotation.npy',
 '/home/jovyan/VU_TEX/playground_Ben/2022-08-26_IVY_GAP/2022-11-09_Subset_optimisation_LBP_v18_patchsize_200/2022-11-09_integer_annotations_binned_patchsize_200/donor_id_292023109_specimen_id_293252159_subimage_id_294219224_annotation.npy',
 '/home/jovyan/VU_TEX/playground_Ben/2022-08-26_IVY_GAP/2022-11-09_Subset_optimisation_LBP_v18_patchsize_200/2022-11-09_integer_annotations_binned_patchsize_200/donor_id_12165_specimen_id_706710_subimage_id_101713024_annotation.npy',
 '/home/jovyan/VU_TEX/playground_Ben/2022-08-26_IVY_GAP/2022-11-09_Subset_optimisation_LBP_v18_patchsize_200/2022-11-09_integer_annotations_binned_patchsize_200/donor_id_15343_specimen_id_281857486_subimage_id_286688001_annotation.npy',
 '/home/jovyan/VU_TEX/playground_Ben/2022-08-26_IVY

In [62]:
#directory_patches_indices = fileroot + 'VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/2021-12-20_Patches/'
#filename_patches_indices = 'HnE_norm_V10S15_patchsize_200_indices_from_1.npy'
#fullpaths_patches_indices = [directory_patches_indices + filename_patches_indices]

In [63]:
directory_out = fileroot + 'VU_TEX/playground_Ben/2022-08-26_IVY_GAP/2022-11-09_Subset_optimisation_LBP_v18_patchsize_200/'
filename_out_base = 'IVY_GAP_subset_v18_'

In [64]:
#directory_images_metadata = fileroot + 'team283_imaging/' + '0ExternalData/2022-09-01_IVY_GAP/'
directory_images_metadata = fileroot + '0ExternalData/2022-09-01_IVY_GAP/'
filename_images_metadata = '2022-09-01_IVY_GAP_metadata.csv'
df_images_metadata = pd.read_csv(directory_images_metadata + filename_images_metadata, index_col=0)
df_images_metadata

,original_index,annotated,output_filename,output_filename_annotations,bits_per_component,data_set_id,expression_x,height,subimage_id,image_height,image_type,image_width,lims1_id,number_of_components,path,resolution,section_number,structure_id_x,tier_count,width,x,y,failed_facet,section_data_set_id,plane_of_section_id,qc_date,section_thickness,specimen_id,sphinx_id_x,storage_directory,weight_x,donor_id,name_x,rna_integrity_number,hemisphere,structure_id_y,parent_id,parent_x_coord,parent_y_coord,parent_z_coord,external_specimen_name,specimen_id_path,is_ish,sphinx_id_y,weight_y,is_cell_specimen,name_y,age_id,organism_id,strain,sex,condition_description,external_donor_name,sex_full_name,tags,multifocal,recurrence_by_six_months,race_only,donor_sex_facet,donor_strain_facet,donor_race_only_facet,donor_condition_description_facet,primary_tissue_source,chemotherapy,egfr_amplification,extent_of_resection,mgmt_methylation,pten_deletion,radiation_therapy,tumor_status,initial_kps,time_to_progression_or_recurrence,mgmt_ihc,survival_days,molecular_subtype
0,0,False,donor_id_15400_specimen_id_268000605_subimage_id_273296711.jpg,donor_id_15400_specimen_id_268000605_subimage_id_273296711_annotation.jpg,8.0,273296709,False,18176,273296711,44736,Primary,79680,NaN,3.0,/external/gbm/prod1/0534341047/0534341047.aff,0.4945,32,9545.0,10,15104,24832,3584,734881840,273296709,8,2014-02-13T20:49:13Z,20.0,268000605,46271,/external/gbm/prod8/image_series_273296709/,5040,15400,H366-0047a.05.01,NaN,left,4084,712742,1,1,1,W40-1-1-E.01,/712737/712742/268000605/,t,512871,9000,f,H366-0047,127,1,unknown,M,"MGMT IHC (0-100), integer) EGFR amplification - Yes MGMT Methylation - No Survival (in days, integer) Radiationtherapy - Yes PTEN deletion - No Initial KPS (0-100 in steps of 10, integer) Extent of resection - Complete Multifocal - No Chemotherapy - Yes Tumor status - primary",W40-1-1,Male,H366-0047,No,NaN,unknown,4122955671,2.904992e+09,2.904992e+09,3275350087,Swedish,Yes,Yes,Complete,No,No,Yes,primary,100,NaN,40.0,184.0,Neural
1,1,False,donor_id_15400_specimen_id_268000605_subimage_id_273296733.jpg,donor_id_15400_specimen_id_268000605_subimage_id_273296733_annotation.jpg,8.0,273296709,NaN,18080,273296733,44736,Primary,77664,NaN,3.0,/external/gbm/prod1/0534340615/0534340615.aff,0.4945,29,NaN,10,15040,23488,3040,734881840,273296709,8,2014-02-13T20:49:13Z,20.0,268000605,46271,/external/gbm/prod8/image_series_273296709/,5040,15400,H366-0047a.05.01,NaN,left,4084,712742,1,1,1,W40-1-1-E.01,/712737/712742/268000605/,t,512871,9000,f,H366-0047,127,1,unknown,M,"MGMT IHC (0-100), integer) EGFR amplification - Yes MGMT Methylation - No Survival (in days, integer) Radiationtherapy - Yes PTEN deletion - No Initial KPS (0-100 in steps of 10, integer) Extent of resection - Complete Multifocal - No Chemotherapy - Yes Tumor status - primary",W40-1-1,Male,H366-0047,No,NaN,unknown,4122955671,2.904992e+09,2.904992e+09,3275350087,Swedish,Yes,Yes,Complete,No,No,Yes,primary,100,NaN,40.0,184.0,Neural
2,2,False,donor_id_15400_specimen_id_268000605_subimage_id_273296751.jpg,donor_id_15400_specimen_id_268000605_subimage_id_273296751_annotation.jpg,8.0,273296709,False,18176,273296751,43968,Primary,79680,NaN,3.0,/external/gbm/prod1/0534340183/0534340183.aff,0.4945,26,9545.0,10,15104,26368,3328,734881840,273296709,8,2014-02-13T20:49:13Z,20.0,268000605,46271,/external/gbm/prod8/image_series_273296709/,5040,15400,H366-0047a.05.01,NaN,left,4084,712742,1,1,1,W40-1-1-E.01,/712737/712742/268000605/,t,512871,9000,f,H366-0047,127,1,unknown,M,"MGMT IHC (0-100), integer) EGFR amplification - Yes MGMT Methylation - No Survival (in days, integer) Radiationtherapy - Yes PTEN deletion - No Initial KPS (0-100 in steps of 10, integer) Extent of resection - Complete Multifocal - No Chemotherapy - Yes Tumor status - primary",W40-1-1,Male,H366-0047,No,NaN,unknown,4122955671,2.904992e+09,2.904992e+09,3275350087,Swedish,Yes,Yes,Complete,No,No,Yes,primary,100,NaN,40.0,184.0,Neural
3,3,False,donor_id_15400_specime

In [65]:
df_images_metadata = df_images_metadata.loc[df_images_metadata['original_index'].isin([50, 594, 406, 418, 
267, 290, 474, 231, 498, 217, 
309, 544])]
df_images_metadata

,original_index,annotated,output_filename,output_filename_annotations,bits_per_component,data_set_id,expression_x,height,subimage_id,image_height,image_type,image_width,lims1_id,number_of_components,path,resolution,section_number,structure_id_x,tier_count,width,x,y,failed_facet,section_data_set_id,plane_of_section_id,qc_date,section_thickness,specimen_id,sphinx_id_x,storage_directory,weight_x,donor_id,name_x,rna_integrity_number,hemisphere,structure_id_y,parent_id,parent_x_coord,parent_y_coord,parent_z_coord,external_specimen_name,specimen_id_path,is_ish,sphinx_id_y,weight_y,is_cell_specimen,name_y,age_id,organism_id,strain,sex,condition_description,external_donor_name,sex_full_name,tags,multifocal,recurrence_by_six_months,race_only,donor_sex_facet,donor_strain_facet,donor_race_only_facet,donor_condition_description_facet,primary_tissue_source,chemotherapy,egfr_amplification,extent_of_resection,mgmt_methylation,pten_deletion,radiation_therapy,tumor_status,initial_kps,time_to_progression_or_recurrence,mgmt_ihc,survival_days,molecular_subtype
50,50,False,donor_id_12106_specimen_id_268416305_subimage_id_273294441.jpg,donor_id_12106_specimen_id_268416305_subimage_id_273294441_annotation.jpg,8.0,273294277,NaN,18080,273294441,44736,Primary,81664,NaN,3.0,/external/gbm/prod1/0534343071/0534343071.aff,0.4945,5,NaN,10,15040,9152,1856,734881840,273294277,7,2013-11-27T18:09:40Z,20.0,268416305,137495,/external/gbm/prod8/image_series_273294277/,5040,12106,H366-0019a.06.A.01,NaN,right,4009,705755,1,1,1,W4-1-1-F.1.01,/705727/705755/268416305/,t,511238,9000,f,H366-0019,79,1,unknown,F,"Survival (in days, integer) Radiationtherapy - Yes Multifocal - No Extent of resection - Complete Chemotherapy - Yes Initial KPS (0-100 in steps of 10, integer) Recurrence by 6 months - Yes Time to progression/recurrence (in days, integer) MGMT Methylation - No MGMT IHC (0-100), integer) PTEN deletion - Yes Tumor status - primary EGFR amplification - No",W4-1-1,Female,H366-0019,No,Yes,unknown,1306038106,2.904992e+09,2.904992e+09,3298293420,Swedish,Yes,No,Complete,No,Yes,Yes,primary,90,102.0,80.0,540.0,"Mesenchymal, Neural"
217,217,True,donor_id_12165_specimen_id_706867_subimage_id_300631309.jpg,donor_id_12165_specimen_id_706867_subimage_id_300631309_annotation.jpg,8.0,300631275,False,18048,300631309,29632,Primary,49792,NaN,3.0,/external/gbm/prod3/0539028775_300197737/0539028775.aff,0.4945,62,NaN,9,15104,5888,5632,734881840,300631275,9,2014-05-05T17:19:49Z,14.0,706867,110034,/external/gbm/prod21/image_series_300631275/,5190,12165,H366-0029a.04.B.05,NaN,right,4132,706198,5,1,1,W12-1-1-D.2.05,/706190/706198/706867/,t,511657,9000,f,H366-0029,86,1,unknown,F,"disease categories - glioblastoma Survival (in days, integer) Time to progression/recurrence (in days, integer) Recurrence by 6 months - unknown Initial KPS (0-100 in steps of 10, integer) Multifocal - No MGMT IHC (0-100), integer) PTEN deletion - No Chemotherapy - No EGFR amplification - Yes Extent of resection - Sub-total MGMT Methylation - No Radiationtherapy - No Tumor status - primary",W12-1-1,Female,H366-0029,No,unknown,unknown,1306038106,2.904992e+09,2.904992e+09,3617011942,Swedish,No,Yes,Sub-total,No,No,No,primary,70,7.0,20.0,80.0,Classical
231,231,False,donor_id_12111_specimen_id_301589924_subimage_id_302049407.jpg,donor_id_12111_specimen_id_301589924_subimage_id_302049407_annotation.jpg,8.0,302049403,False,18048,302049407,29376,Primary,49792,NaN,3.0,/external/gbm/prod4/0539000884_301628602/0539000884.aff,0.4994,294,NaN,9,15104,8064,5632,734881840,302049403,7,2014-07-18T20:42:17Z,14.0,301589924,10192,/external/gbm/prod21/image_series_302049403/,5190,12111,H366-0025a.05.B.03,NaN,left,4084,705834,3,1,1,W10-1-1-E.2.03,/705810/705834/301589924/,t,511394,9000,f,H366-0025,42,1,unknown,F,"disease categories - glioblastoma EGFR amplification - No Recurrence by 6 months - unknown Chemotherapy - Yes Extent of resection - Complete MGMT Methylation - Yes Radiationtherapy - Yes MGMT IHC (0-100), integer) Initial KPS (

In [66]:
#directory_colors_metadata = fileroot + 'team283_imaging/' + '0ExternalData/2022-09-01_IVY_GAP/'
directory_colors_metadata = fileroot + '0ExternalData/2022-09-01_IVY_GAP/'
filename_colors_metadata = '2022-09-02_colors_for_IVY_GAP.csv'
df_colors = pd.read_csv(directory_colors_metadata + filename_colors_metadata, index_col=0)
df_colors

,color_indices,colors_hex,colors_rgb255,region_names,official_region_names,colorblind_safe,colorblind_optimized
0,0,#FFFFFF,"(255, 255, 255)",Background,Bg,#800080,#ff9999
1,1,#218FA5,"(33, 143, 165)",Leading Edge,LEregion,#218FA5,#006400
2,2,#D104D0,"(209, 4, 208)",Infiltrating Tumor,ITregion,#D104D0,#ff1493
3,3,#05D004,"(5, 208, 4)",Cellular Tumor,CTregion,#05D004,#a6730a
4,4,#FF6600,"(255, 102, 0)",Hyperplastic blood vessels,LE_IT_CT_hbv,#FF6600,#ff8c00
5,5,#43D1F8,"(67, 209, 248)",Perinecrotic zone,CTpnz,#43D1F8,#00bfff
6,6,#0504D0,"(5, 4, 208)","Pseudopalisading cells, no necrosis",CTpnn,#0504D0,#0000ff
7,7,#05D0AA,"(5, 208, 170)","Pseudopalisading cells, around necrosis",CTpan,#05D0AA,#ffd700
8,8,#FF3300,"(255, 51, 0)",Microvascular proliferation,CTmvp,#006400,#800080
9,9,#060505,"(6, 5, 5)",Necrosis,CTne,#060505,#000054


In [67]:
# make list of methods
methods_list = os.listdir(directory_lbp_top)
methods_list.sort(key=partial(uf.getvaluefromstringbest, variable='radius', mydtype=int))
methods_list.sort(key=partial(uf.getvaluefromstringbest, variable='Ch', mydtype=int))
methods_list = [each for each in methods_list if
               uf.getvaluefromstringbest(each, variable='radius', mydtype=int) < 200]
method_list_cols = []
for each in methods_list:
    print(each)
    this_npoints = uf.getvaluefromstringbest(each, variable='npoints', mydtype=int)
    for i in range(0, this_npoints+2):
        method_list_cols.append(each+'_value_'+str(i))
    print(this_npoints)
print(method_list_cols)

Ch_0_METHOD_uniform_npoints_7_radius_1
7
Ch_0_METHOD_uniform_npoints_13_radius_2
13
Ch_0_METHOD_uniform_npoints_19_radius_3
19
Ch_0_METHOD_uniform_npoints_26_radius_4
26
Ch_0_METHOD_uniform_npoints_32_radius_5
32
Ch_0_METHOD_uniform_npoints_38_radius_6
38
Ch_0_METHOD_uniform_npoints_51_radius_8
51
Ch_0_METHOD_uniform_npoints_70_radius_11
70
Ch_0_METHOD_uniform_npoints_88_radius_14
88
Ch_0_METHOD_uniform_npoints_120_radius_19
120
Ch_0_METHOD_uniform_npoints_158_radius_25
158
Ch_0_METHOD_uniform_npoints_214_radius_34
214
Ch_0_METHOD_uniform_npoints_283_radius_45
283
Ch_0_METHOD_uniform_npoints_371_radius_59
371
Ch_0_METHOD_uniform_npoints_497_radius_79
497
Ch_0_METHOD_uniform_npoints_654_radius_104
654
Ch_0_METHOD_uniform_npoints_874_radius_139
874
Ch_0_METHOD_uniform_npoints_1157_radius_184
1157
Ch_1_METHOD_uniform_npoints_7_radius_1
7
Ch_1_METHOD_uniform_npoints_13_radius_2
13
Ch_1_METHOD_uniform_npoints_19_radius_3
19
Ch_1_METHOD_uniform_npoints_26_radius_4
26
Ch_1_METHOD_uniform_npoi

In [68]:
len(method_list_cols)

9416

In [69]:
overall_number_of_superpixels = 0
for height, width in zip(df_images_metadata['image_height'], df_images_metadata['image_width']):
    height_sp = math.floor(height/patchsize)
    width_sp = math.floor(width/patchsize)
    overall_number_of_superpixels += height_sp*width_sp
overall_number_of_superpixels

953894

In [70]:
anndata_list = []
anndata_concat = ad.AnnData()
out_array = np.zeros((overall_number_of_superpixels, len(method_list_cols)), dtype=np.uint16)
obs_list = []
m = 0
n = 0

for j, output_filename, output_fname_annotated in tqdm(zip(df_images_metadata['original_index'], 
                                                      df_images_metadata['output_filename'],
                                                     df_images_metadata['output_filename_annotations']),
                                                      total = len(df_images_metadata)):
#    print(j, output_filename, output_fname_annotated)
    
    fpath_for_shape = directory_lbp_top + methods_list[0] + '/' + output_filename.replace('.jpg', '.npy')
    this_test_array = np.load(fpath_for_shape, mmap_mode='r')
    this_shape = this_test_array.shape
    this_dtype = this_test_array.dtype
    
    #this part creates the X0 and X1 coordinates
    x0_array = np.zeros((this_shape[0], this_shape[1]), dtype=np.uint16)
    for i0 in range(this_shape[0]):
        x0_array[i0] = i0
    
    x1_array = np.zeros((this_shape[0], this_shape[1]), dtype=np.uint16)
    for i1 in range(this_shape[1]):
        x1_array[:, i1] = i1
        
    #loading the groundtruth
    gt = np.load(directory_gt + output_fname_annotated.replace('.jpg', '.npy'))
    
    output_array = np.zeros((this_shape[0], this_shape[1], len(method_list_cols)), dtype=this_dtype)
#    print(this_shape, this_dtype, output_array.shape)
    
    start_index = 0
    for each_method in methods_list:
        fpath_to_add = directory_lbp_top + each_method + '/' + output_filename.replace('.jpg', '.npy')
        this_npoints = uf.getvaluefromstringbest(each_method, variable='npoints', mydtype=int)
        array_to_add = np.load(fpath_to_add)
        end_index = start_index + this_npoints + 2
        #add to the array
#        print(fpath_to_add, start_index, end_index, array_to_add.shape)
        output_array[:, :, start_index:end_index] = array_to_add
        start_index = end_index
    
    #this part reshapes the arrays
    reshaped = np.reshape(output_array, (-1, output_array.shape[2]))
    reshaped_x0 = np.reshape(x0_array, (-1))
    reshaped_x1 = np.reshape(x1_array, (-1))
    reshaped_gt = np.reshape(gt, (-1))
        
    this_AD = ad.AnnData(reshaped, var=method_list_cols, dtype=np.uint16)
    
    this_AD.obs['this_image_index'] = this_AD.obs.index
    this_AD.obs['X0'] = reshaped_x0 
    this_AD.obs['X1'] = reshaped_x1 
    this_AD.obs['Groundtruth'] = reshaped_gt
    this_AD.obs['original_index'] = j
    this_AD.obs['output_filename'] = output_filename
    this_AD.obs['output_fname_annotated'] = output_fname_annotated
    
#    anndata_list.append(this_AD)
#    if m == 0:
#        anndata_concat = this_AD
#    else:
#        anndata_concat = ad.concat((anndata_concat, this_AD), axis=0)
#    break
    this_shape = this_AD.X.shape
    out_array[n:n+this_shape[0]] = this_AD.X
    print(n, n+this_shape[0])
    n+=this_shape[0]
    
    obs_list.append(this_AD.obs)

    m+=1

  0%|          | 0/12 [00:00<?, ?it/s]

0 6750
6750 13500
13500 20250
20250 27000
27000 33750
33750 40500
40500 47250
47250 53540
53540 60290
60290 67040
67040 73790
73790 80540


In [71]:
n

80540

In [72]:
out_array[n-1]

array([    0,     0,     6, ...,     0,     0, 36630], dtype=uint16)

In [73]:
out_array[n]

array([0, 0, 0, ..., 0, 0, 0], dtype=uint16)

In [74]:
out_array_shortened = out_array[0:n]
out_array_shortened

array([[    0,     0,     1, ...,     0, 40000,     0],
       [    0,     0,     0, ...,     0, 39622,   306],
       [   67,    51,   310, ...,     0,   808, 36390],
       ...,
       [    0,     0,     0, ...,     0,     0, 39092],
       [    0,     0,     0, ...,     4,    69, 38655],
       [    0,     0,     6, ...,     0,     0, 36630]], dtype=uint16)

In [75]:
#anndata_concat.X.shape

In [76]:
start = datetime.now(); print(start)
obs_concat = pd.concat(obs_list, ignore_index=True)
print(datetime.now()-start)

2022-11-10 09:47:30.375748
0:00:00.011318


In [77]:
obs_concat

,this_image_index,X0,X1,Groundtruth,original_index,output_filename,output_fname_annotated
0,0,0,0,0,50,donor_id_12106_specimen_id_268416305_subimage_id_273294441.jpg,donor_id_12106_specimen_id_268416305_subimage_id_273294441_annotation.jpg
1,1,0,1,0,50,donor_id_12106_specimen_id_268416305_subimage_id_273294441.jpg,donor_id_12106_specimen_id_268416305_subimage_id_273294441_annotation.jpg
2,2,0,2,0,50,donor_id_12106_specimen_id_268416305_subimage_id_273294441.jpg,donor_id_12106_specimen_id_268416305_subimage_id_273294441_annotation.jpg
3,3,0,3,0,50,donor_id_12106_specimen_id_268416305_subimage_id_273294441.jpg,donor_id_12106_specimen_id_268416305_subimage_id_273294441_annotation.jpg
4,4,0,4,0,50,donor_id_12106_specimen_id_268416305_subimage_id_273294441.jpg,donor_id_12106_specimen_id_268416305_subimage_id_273294441_annotation.jpg
...,...,...,...,...,...,...,...
80535,6745,89,70,3,594,donor_id_13988_specimen_id_278113164_subimage_id_286689303.jpg,donor_id_13988_specimen_id_278113164_subimage_id_286689303_annotation.jpg
80536,6746,89,71,0,594,donor_id_13988_specimen_id_278113164_subimage_id_286689303.jpg,donor_id_13988_specimen_id_278113164_subimage_id_286689303_annotation.jpg
80537,6747,89,72,0,594,donor_id_13988_specimen_id_278113164_subimage_id_286689303.jpg,donor_id_13988_specimen_id_278113164_subimage_id_286689303_annotation.jpg
80538,6748,89,73,0,594,donor_id_13988_specimen_id_278113164_subimage_id_286689303.jpg,donor_id_13988_specimen_id_278113164_subimage_id_286689303_annotation.jpg


In [78]:
#anndata_list[0].X

In [79]:
#anndata_list[1].X

In [80]:
#anndata_concat = ad.concat(anndata_list, axis=0, index_unique='-')
#anndata_concat.obs.reset_index(inplace=True, drop=True)
#anndata_concat.obs.index = anndata_concat.obs.index.astype(str)
#anndata_concat.obs = pd.DataFrame(anndata_concat.obs.index, columns=['Here'])
#anndata_concat.obs.index = anndata_concat.obs.index.astype(str)
#anndata_concat.obs['output_filename'] = anndata_concat.obs['output_filename'].astype(str)
#anndata_concat.var = pd.DataFrame(method_list_cols)
#display(anndata_concat)
#display(anndata_concat.var)
#display(anndata_concat.obs)

In [81]:
ad.__version__

'0.8.0'

In [82]:
start = datetime.now(); print(start)
np.save("2022-11-09_LBP_all_images_data_X.npy", out_array_shortened)
print(datetime.now()-start)

2022-11-10 09:47:30.452683
0:00:03.775988


In [83]:
obs_concat.to_csv("2022-11-09_LBP_all_images_data_OBS.csv")

In [84]:
pd.DataFrame(method_list_cols).to_csv("2022-11-09_LBP_all_images_data_VAR.csv")

In [85]:
#anndata_concat.write("2022-10-11_LBP_all_images_data.h5ad")